<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Exploring_julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'
export JULIA_NUM_THREADS=2

After you run the first cell (the the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU).

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [1]:
VERSION

v"1.7.2"

The following piece of code was written by Dr. William Godoy from ORNL.
The complete code is available in his github repository: https://github.com/williamfgc/simple-gemm

This initial version is a sequential version of matrix multiplication


In [2]:
import Random

@doc """
Simplified gemm: C = alpha A x B + C where alpha = 1 , C = zeros,
so:
C = A x B
"""
function gemm!(A::Array{Float32,2}, B::Array{Float32,2}, C::Array{Float32,2})

    A_rows = size(A)[1]
    A_cols = size(A)[2]
    B_cols = size(B)[2]

     for j = 1:B_cols
        for l = 1:A_cols
            @inbounds temp::Float32 = B[l, j]::Float32
            for i = 1:A_rows
                @inbounds C[i, j] += temp * A[i, l]
            end
        end
    end
end

function main(args::Array{String,1})::Int32

    # must initialize scalars
    A_rows::Int64 = -1
    A_cols::Int64 = -1
    B_rows::Int64 = -1
    B_cols::Int64 = -1

    steps::Int32 = 1

    @show args

    # args don't include Julia executable and program
    nargs = size(args)[1]

    if nargs == 4 || nargs == 3
        A_rows = parse(Int64, args[1])
        A_cols = parse(Int64, args[2])
        B_rows = parse(Int64, args[2])
        B_cols = parse(Int64, args[3])

        if nargs == 4
            steps = parse(Int32, args[4])
        end
    else
        throw(
            ArgumentError( "Usage: \n - 3 arguments: matrix A rows, matrix A cols and matrix B cols\n- 4 arguments: matrix A rows, matrix A cols and matrix B cols, steps",
            ),
        )
    end

    # Julia is column-based (like Fortran)
    @time begin

        print("Time to allocate A ")
        @time A = Array{Float32,2}(undef, A_rows, A_cols)
        print("Time to allocate B ")
        @time B = Array{Float32,2}(undef, B_rows, B_cols)
        print("Time to initialize C ")
        @time C = zeros(Float32, A_rows, B_cols)

        print("Time to fill A ")
        @time Random.rand!(A)
        print("Time to fill B ")
        @time Random.rand!(B)

        print("Time to simple gemm ")
        @time gemm!(A, B, C)

        if steps > 1
            timings = zeros(steps)
            for i = 2:steps
                print("Time to simple gemm ")
                 timings[i] = @elapsed gemm!(A, B, C)
                println(timings[i])
            end

            average_time = sum(timings) / (steps - 1)
            gflops = (2 * A_rows * A_cols * B_cols) * 1E-9 / average_time
            println(
                "GFLOPS: ",
                gflops,
                " steps: ",
                steps,
                " average_time: ",
                average_time,
            )
        end

        print("Time to total time ")
    end
    return 0

end



main(["500","500","500","5"])

args = ["500", "500", "500", "5"]
Time to allocate A   0.000004 seconds (2 allocations: 976.609 KiB)
Time to allocate B   0.000019 seconds (2 allocations: 976.609 KiB)
Time to initialize C   0.000660 seconds (2 allocations: 976.609 KiB)
Time to fill A   0.000147 seconds
Time to fill B   0.000715 seconds
Time to simple gemm   0.026087 seconds
Time to simple gemm 0.025073825
Time to simple gemm 0.026045731
Time to simple gemm 0.024597237
Time to simple gemm 0.025052874
GFLOPS: 9.923621162705638 steps: 5 average_time: 0.025192416750000002
Time to total time   0.329998 seconds (116.96 k allocations: 9.448 MiB, 4.82% gc time, 60.71% compilation time)


0

This second version uses Thread parallelization.
Something similar to using OpenMP parallel loop or NUMBA's prange

In [3]:
Threads.nthreads()

1

In [4]:
import Random

@doc """
Simplified gemm: C = alpha A x B + C where alpha = 1 , C = zeros,
so:
C = A x B
"""
function gemm!(A::Array{Float32,2}, B::Array{Float32,2}, C::Array{Float32,2})

    A_rows = size(A)[1]
    A_cols = size(A)[2]
    B_cols = size(B)[2]

    Threads.@threads for j = 1:B_cols
        for l = 1:A_cols
            @inbounds temp::Float32 = B[l, j]::Float32
            for i = 1:A_rows
                @inbounds C[i, j] += temp * A[i, l]
            end
        end
    end
end

function main(args::Array{String,1})::Int32

    # must initialize scalars
    A_rows::Int64 = -1
    A_cols::Int64 = -1
    B_rows::Int64 = -1
    B_cols::Int64 = -1

    steps::Int32 = 1

    @show args

    # args don't include Julia executable and program
    nargs = size(args)[1]

    if nargs == 4 || nargs == 3
        A_rows = parse(Int64, args[1])
        A_cols = parse(Int64, args[2])
        B_rows = parse(Int64, args[2])
        B_cols = parse(Int64, args[3])

        if nargs == 4
            steps = parse(Int32, args[4])
        end
    else
        throw(
            ArgumentError( "Usage: \n - 3 arguments: matrix A rows, matrix A cols and matrix B cols\n- 4 arguments: matrix A rows, matrix A cols and matrix B cols, steps",
            ),
        )
    end

    # Julia is column-based (like Fortran)
    @time begin

        print("Time to allocate A ")
        @time A = Array{Float32,2}(undef, A_rows, A_cols)
        print("Time to allocate B ")
        @time B = Array{Float32,2}(undef, B_rows, B_cols)
        print("Time to initialize C ")
        @time C = zeros(Float32, A_rows, B_cols)

        print("Time to fill A ")
        @time Random.rand!(A)
        print("Time to fill B ")
        @time Random.rand!(B)

        print("Time to simple gemm ")
        @time gemm!(A, B, C)

        if steps > 1
            timings = zeros(steps)
            for i = 2:steps
                print("Time to simple gemm ")
                 timings[i] = @elapsed gemm!(A, B, C)
                println(timings[i])
            end

            average_time = sum(timings) / (steps - 1)
            gflops = (2 * A_rows * A_cols * B_cols) * 1E-9 / average_time
            println(
                "GFLOPS: ",
                gflops,
                " steps: ",
                steps,
                " average_time: ",
                average_time,
            )
        end

        print("Time to total time ")
    end
    return 0

end



main(["500","500","500","5"])

args = ["500", "500", "500", "5"]
Time to allocate A   0.000023 seconds (2 allocations: 976.609 KiB)
Time to allocate B   0.000006 seconds (2 allocations: 976.609 KiB)
Time to initialize C   0.000488 seconds (2 allocations: 976.609 KiB)
Time to fill A   0.000406 seconds
Time to fill B   0.000527 seconds
Time to simple gemm   0.077795 seconds (63.66 k allocations: 3.463 MiB, 78.08% compilation time)
Time to simple gemm 0.017281473
Time to simple gemm 0.016447019
Time to simple gemm 0.017067844
Time to simple gemm 0.016816876
GFLOPS: 14.790008792956026 steps: 5 average_time: 0.016903303
Time to total time   0.147348 seconds (64.23 k allocations: 6.348 MiB, 41.22% compilation time)


0

**The next three cells are for GPU benchmarking. If you are using this notebook for the first time and have GPU enabled, you can give it a try.**

### Optional GPU Experiments

In [ ]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])
using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

The GPU device is:CuDevice(0)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed GPUArraysCore ────────── v0.1.5
   Installed IrrationalConstants ──── v0.2.2
   Installed CUDA_Driver_jll ──────── v0.5.0+1
   Installed Adapt ────────────────── v3.7.2
   Installed Scratch ──────────────── v1.2.1
   Installed BenchmarkTools ───────── v1.5.0
   Installed SpecialFunctions ─────── v2.3.1
   Installed CUDA_Runtime_Discovery ─ v0.2.4
   Installed TimerOutputs ─────────── v0.5.23
   Installed StaticArraysCore ─────── v1.4.2
   Installed AbstractFFTs ─────────── v1.5.0
   Installed GPUCompiler ──────────── v0.21.4
   Installed LLVMExtra_jll ────────── v0.0.23+0
   Installed UnsafeAtomicsLLVM ────── v0.1.3
   Installed StaticArrays ─────────── v1.9.3
   Installed CEnum ────────────────── v0.4.2
   Installed CUDA_Runtime_jll ─────── v0.6.0+0
   Installed BFloat16s ────────────── v0.4.2
   Installed Reexport ─────────────── v1.2.2
   Installed Random123 ────────────── v1.6.

In [ ]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

BenchmarkTools.Trial: 126 samples with 1 evaluation.
 Range (min … max):  35.569 ms … 74.292 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     38.148 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.703 ms ±  5.308 ms  ┊ GC (mean ± σ):  0.80% ± 2.28%

  ▁▅ ▁▆▄█                                                      
  ██▇█████▃▆▃▆▃▆▅▇▁▅▃▄▃▄▁▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▃▁▁▁▁▁▁▁▃▁▁▁▃▁▁▃▁▁▁▁▃ ▃
  35.6 ms         Histogram: frequency by time        60.3 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

In [ ]:
println("The CuArrray operation should be much faster (~100 times) than the CPU implemenation.")
mgpu = cu(mcpu)
@benchmark CUDA.@sync mgpu*mgpu

The CuArrray operation should be much faster (~100 times) than the CPU implemenation.


BenchmarkTools.Trial: 7610 samples with 1 evaluation.
 Range (min … max):  469.377 μs …  19.478 ms  ┊ GC (min … max): 0.00% … 48.84%
 Time  (median):     632.475 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   647.089 μs ± 335.904 μs  ┊ GC (mean ± σ):  0.37% ±  0.77%

                     ▃▆▇██▆▄▂                                    
  ▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▆█████████▇▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂ ▃
  469 μs           Histogram: frequency by time          906 μs <

 Memory estimate: 912 bytes, allocs estimate: 41.

In [ ]:
a = 10
b = 20
c = a + b

30

In [ ]:
function add(a,b)
  return a+b
end

add (generic function with 1 method)

In [ ]:
c = add(a,b)

30

In [ ]:
x = 3.1416
y = 1.4142
z = add(x,y)

4.5558